# XGBoost

Lo que hace tan popular a XGBoost es la velocidad y los resultados que logra alcanzar. El algoritmos es paralelizable y por ende logra ser rápido de entrenar, se puede paralelizar en la GPU y entre una red de computadores

XGBoost ha logrado una performance al nivel del estado del arte en muchas tareas de ML

### 1- Clasificación con XGBoost

In [8]:
import xgboost as xgb
import pandas as pd
import numpy as np
import warnings
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
warnings.filterwarnings("ignore")

Usamos el dataset de cancer de mamas (de ML con árboles de decisión) para crear un modelo de XGBoost rápido

In [9]:
# Traemos los datos
wbc = pd.read_csv("datasets/MLTreeModels/wbc.csv")
# Reemplazamos por 1 y 0
wbc['diagnosis'] = wbc['diagnosis'].replace(['M', 'B'], [1, 0])
X = wbc[['radius_mean', 'texture_mean', 'perimeter_mean',
         'area_mean', 'smoothness_mean', 'compactness_mean',
         'concavity_mean', 'concave points_mean', 'symmetry_mean',
         'fractal_dimension_mean', 'radius_se', 'texture_se',
         'perimeter_se', 'area_se', 'smoothness_se',
         'compactness_se', 'concavity_se', 'concave points_se',
         'symmetry_se', 'fractal_dimension_se', 'radius_worst',
         'texture_worst', 'perimeter_worst', 'area_worst',
         'smoothness_worst', 'compactness_worst', 'concavity_worst',
         'concave points_worst', 'symmetry_worst',
         'fractal_dimension_worst']]
y = wbc[["diagnosis"]]

# Dividimos los datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                   random_state=123)

# Instanciamos el objeto
xg_cl = xgb.XGBClassifier(objective="binary:logistic",
                          n_estimators=10, seed=123)

# Ajustamos a los datos de entrenamiento
xg_cl.fit(X_train, y_train)

# Predecimos
preds = xg_cl.predict(X_test)

# Evaluamos
score = accuracy_score(y_test, preds)
print(score)

0.956140350877193


XGBoost es un ensamble algorithm ya que usa el resultado de muchos otros modelos para poder predecir. Usa los árboles de decisión como base learners

Boosting se puede entender como un concepto que se aplica a un set de modelos de machine learning, es un algoritmo de ensamble. Se usa para converit un conjunto de weak learners en un strong learner (cualquier algoritmo que pueda ser perfeccionado para lograr una mejor performance)

Cross-validation en XGBoost

Para usar cross-validation directamente con xgboost debemos transformar la data en un formato especifico llamado DMatrix

In [3]:
import xgboost as xgb
import pandas as pd


# Traemos los datos
wbc = pd.read_csv("datasets/MLTreeModels/wbc.csv")
# Reemplazamos por 1 y 0
wbc['diagnosis'] = wbc['diagnosis'].replace(['M', 'B'], [1, 0])
X = wbc[['radius_mean', 'texture_mean', 'perimeter_mean',
         'area_mean', 'smoothness_mean', 'compactness_mean',
         'concavity_mean', 'concave points_mean', 'symmetry_mean',
         'fractal_dimension_mean', 'radius_se', 'texture_se',
         'perimeter_se', 'area_se', 'smoothness_se',
         'compactness_se', 'concavity_se', 'concave points_se',
         'symmetry_se', 'fractal_dimension_se', 'radius_worst',
         'texture_worst', 'perimeter_worst', 'area_worst',
         'smoothness_worst', 'compactness_worst', 'concavity_worst',
         'concave points_worst', 'symmetry_worst',
         'fractal_dimension_worst']]
y = wbc[["diagnosis"]]
churn_dmatrix = xgb.DMatrix(data=X, label=y)

params = {"objective": "binary:logistic", "max_depth":4}

cv_results = xgb.cv(dtrain=churn_dmatrix, params=params, nfold=4,
                    num_boost_round=10, metrics="error", as_pandas=True)

performance = round((1 - cv_results["test-error-mean"]).iloc[-1], 4)
print(f"Accuracy: {performance}")

Accuracy: 0.9525


XGBoost es útil cuando se tienen grandes cantidades de datos (aunque basta con el numero de caracteristicas sea menor al numero de filas)

Funciona bien cuando hay una mezcla de variables categoricas y numericas, o cuando son solo numericas

No es un buen algoritomo cuandos se trata de reconocimiento de imagenes, computer vision o NLP (para lo que es mejor usar Deep Learning)

### 2- Regresión con XGBoost

Predecir valores continuos, en la mayoría de los casos se usa RMSE como métrica para medire la performanca, aunque tambien se usa frecuentemente MAE

#### 2.1 Funciones objetivo

Miden que tan lejos estan las predicciones de los valores reales, el objetivo es encontrar el modelo que minimice el valor de la funcion objetivo

En XGBoost las funciones objetivo mas comunes son reg:linear (para problemas de regresión), reg:logistic (cuando se busca la clasificación de un registro) y binary:logistic (cuando se quiere saber la probabilidad de un suceso)

#### 2.1 Base Learners

XGBoost combina muchos modelos individuales para poder predecir, cada uno de esos modelos individuales se conoce como base learner. Lo que se busca es que estos base learner sean un poco mejor que un modelo aleatorio en predecir una parte especifica del dataset y uniformamente malos en predecir lo demás

In [2]:
import xgboost as xgb
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

boston_data = pd.read_csv("datasets/XGBoost/housing.csv")
boston_data.head()

,RM,LSTAT,PTRATIO,MEDV
0,6.575,4.98,15.3,504000.0
1,6.421,9.14,17.8,453600.0
2,7.185,4.03,17.8,728700.0
3,6.998,2.94,18.7,701400.0
4,7.147,5.33,18.7,760200.0


In [11]:
X, y = boston_data.iloc[:, :-1], boston_data.iloc[:,-1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                    random_state=123)

xg_reg = xgb.XGBRegressor(objective="reg:squarederror", n_estimators=10,
                          seed=123)

xg_reg.fit(X_train, y_train)

preds = xg_reg.predict(X_test)

In [12]:
from sklearn.metrics import mean_squared_error

rmse = np.sqrt(mean_squared_error(y_test, preds))
print(f"RMSE: {rmse}")

RMSE: 64703.978919794106
